In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import boto3
import io
from datetime import date

In [7]:
pd.__version__

'1.0.1'

In [5]:
pd.set_option('display.max_columns', None)

bucketname = 'lems-spot-geodata-dev'
data_folder = 'data/raw'
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucketname)
today = str(date.today())
print(today)
directory_name = today
prefix_objs = bucket.objects.filter(Prefix=f"{data_folder}/{directory_name}/denue")
print(prefix_objs)

prefix_df = []

for obj in prefix_objs:
    key = obj.key
    body = obj.get()['Body'].read()
    print(f'Reading {key}')
    temp = pd.read_csv(io.BytesIO(body), encoding='ISO-8859-1', low_memory=False)
    print(temp.info())


2022-05-24
s3.Bucket.objectsCollection(s3.Bucket(name='lems-spot-geodata-dev'), s3.ObjectSummary)
Reading data/raw/2022-05-24/denue_1.csv


TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type